# Download Data

```{admonition} WIP
This notebook is currently a work in progress, meaning it likely has incomplete explanations and some non-functional code/links/ect.  Please be patient!
```

Automatically download data from the AMISR Website.

Author: Doga Ozturk

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import sys
sys.path
import os
import datetime as dt
import pandas as pd
import matplotlib.dates as mdates
myFmt = mdates.DateFormatter('%H:%M')
import h5py
import requests

/Users/dsozturk/Library/Python/3.8/lib/python/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
def find_exp_extension(exp_link):
    ext_list = []
    for i in range(10):
        try_link = exp_link+'.00{0:d}/'.format(i)
        response = requests.get(try_link)
        if response.status_code == 200:
            ext = i
        else:
            ext = 404
            print('No '+try_link+' experiment at this time.')
        ext_list.append(ext)
    return ext_list

In [3]:
def find_lp_time_res(exp_link, exp_time, ext_list):
    lp_data_links = []
    vvels_data_links = []
    
    for ext in ext_list:
        lp_data_base = 'https://urldefense.us/v3/__https://data.amisr.com/database/dbase_site_media/PFISR/Experiments/*7B0:*Y*m*d*7D'.format(exp_time)*'.00*7B0:d*7D'.format(ext)*'/DataFiles/*7B0:*Y*m*d*7D'.format(exp_time)*'.00*7B0:d*7D'.format(ext)*'_lp_'*5Cn__;JSUlJSUrJSUrJSUlJSUrJSUrJQ!!Nv3xtKNH_4uope0!hjObzLJgdlF68DnBdXk7s2eah_BPzOV8ksVsnwer_O6lgna7o3h5qFJwMAP6x1Lc7MPD5SgZ9qJaYEFCDlGUzg$         lp_data_ext = 'min-cal.h5'
        time_steps = [1, 3, 5, 10]
        for res in time_steps:
            lp_data_link = lp_data_base + '{0:d}'.format(res) + lp_data_ext
            response = requests.get(lp_data_link)
            if response.status_code == 200:
                print('Time resolution for LP is ', res)
                vvels_data_link = lp_data_link[:-3]+'-vvelsLat-{0:d}'.format(int(res*60))+'sec.h5'
                response_vvels = requests.get(vvels_data_link)
                if response_vvels.status_code == 200:
                    print('VVels for experiment found.')
                else:
                    vvels_data_link = 404
                break
            else:
                lp_data_link = 404
                vvels_data_link = 404
        lp_data_links.append(lp_data_link)
        vvels_data_links.append(vvels_data_link)
        
    return lp_data_links, vvels_data_links
    

In [4]:
def find_ac_time_res(exp_link, exp_time, ext_list):
    ac_data_links = []
    for ext in ext_list:
        ac_data_base = 'https://urldefense.us/v3/__https://data.amisr.com/database/dbase_site_media/PFISR/Experiments/*7B0:*Y*m*d*7D'.format(exp_time)*'.00*7B0:d*7D'.format(ext)*'/DataFiles/*7B0:*Y*m*d*7D'.format(exp_time)*'.00*7B0:d*7D'.format(ext)*'_ac_'*5Cn__;JSUlJSUrJSUrJSUlJSUrJSUrJQ!!Nv3xtKNH_4uope0!hjObzLJgdlF68DnBdXk7s2eah_BPzOV8ksVsnwer_O6lgna7o3h5qFJwMAP6x1Lc7MPD5SgZ9qJaYEEvxqxgyg$         ac_data_ext = 'min-cal.h5'
        time_steps = [1, 3, 5, 10]
        for res in time_steps:
            ac_data_link = ac_data_base + '{0:d}'.format(res) + ac_data_ext
            response = requests.get(ac_data_link)
            if response.status_code == 200:
                print('Time resolution for AC is ', res)
                break
            else:
                ac_data_link = 404
        ac_data_links.append(ac_data_link)
    return ac_data_links

In [5]:
conjunctions = pd.read_csv('./THEMIS_PFISR_conjunctions.csv')
conjunctions.StartTime = pd.to_datetime(conjunctions.StartTime)
conjunctions.StopTime = pd.to_datetime(conjunctions.StopTime)

In [6]:
conjunctions["LP_data"] = 'no'
conjunctions["VVels_data"] = 'no'
conjunctions["AC_data"] = 'no'

In [7]:
base_url = 'https://urldefense.us/v3/__https://data.amisr.com/database/61/experiment/__;!!Nv3xtKNH_4uope0!hjObzLJgdlF68DnBdXk7s2eah_BPzOV8ksVsnwer_O6lgna7o3h5qFJwMAP6x1Lc7MPD5SgZ9qJaYEHjlyq1ag$ '

In [ ]:
lp_link_list = []
ac_link_list = []
vvels_link_list = []

for i in range(conjunctions.shape[0]):
    exp_link = base_url + '{0:%Y%m%d}'.format(conjunctions.StartTime.iloc[i])
    ext_list = find_exp_extension(exp_link)
    lp_links, vvels_links = find_lp_time_res(exp_link, conjunctions.StartTime.iloc[i], ext_list)
    ac_links = find_ac_time_res(exp_link, conjunctions.StartTime.iloc[i], ext_list)
    
    if lp_links != 404:
        conjunctions.LP_data.iloc[i] = 'yes'
    if vvels_links != 404:
        conjunctions.VVels_data.iloc[i] = 'yes'
    if ac_links != 404:
        conjunctions.AC_data.iloc[i] = 'yes'
        
    lp_links.append(lp_link)
    ac_links.append(ac_link)
    vvels_links.append(vvels_link)

No https://urldefense.us/v3/__https://data.amisr.com/database/61/experiment/20121003.000/__;!!Nv3xtKNH_4uope0!hjObzLJgdlF68DnBdXk7s2eah_BPzOV8ksVsnwer_O6lgna7o3h5qFJwMAP6x1Lc7MPD5SgZ9qJaYEExFofbvA$  experiment at this time.
No https://urldefense.us/v3/__https://data.amisr.com/database/61/experiment/20121003.004/__;!!Nv3xtKNH_4uope0!hjObzLJgdlF68DnBdXk7s2eah_BPzOV8ksVsnwer_O6lgna7o3h5qFJwMAP6x1Lc7MPD5SgZ9qJaYEEnc0g7Qw$  experiment at this time.
No https://urldefense.us/v3/__https://data.amisr.com/database/61/experiment/20121003.005/__;!!Nv3xtKNH_4uope0!hjObzLJgdlF68DnBdXk7s2eah_BPzOV8ksVsnwer_O6lgna7o3h5qFJwMAP6x1Lc7MPD5SgZ9qJaYEGSEriGDw$  experiment at this time.
No https://urldefense.us/v3/__https://data.amisr.com/database/61/experiment/20121003.006/__;!!Nv3xtKNH_4uope0!hjObzLJgdlF68DnBdXk7s2eah_BPzOV8ksVsnwer_O6lgna7o3h5qFJwMAP6x1Lc7MPD5SgZ9qJaYEGNphBMvg$  experiment at this time.
No https://urldefense.us/v3/__https://data.amisr.com/database/61/experiment/20121003.007/__;!!Nv3xtKNH_4

In [ ]:
vvels_links != 404

In [ ]:
conjunctions